## Testing tool for downloading, unziping, and cogging CHIRPS Africa Monthly product

In [ ]:
import os
import gzip
import urllib
import rasterio
import xarray as xr
from datacube.utils.cog import write_cog
from datacube.utils.geometry import assign_crs

In [ ]:
def download_and_cog_chirps(year, month, s3_dst):
    
    #set up file strings
    filename = f"chirps-v2.0.{year}.{month}.tif"
    out_filename = f"{s3_dst}/chirps-v2.0_{year}.{month}.tif"
    url = f"https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_monthly/tifs/{filename}"
    
    # download url
    try:
        if not os.path.exists(out_filename):
            response = urllib.request.urlopen(url)
    
    
        #unzip
        with gzip.GzipFile(fileobj=response, mode='rb') as gzip_infile:

            #grab tif data and metadata
            with rasterio.open(gzip_infile, 'r') as src:
                raster=src.read()
                transform=src.transform
                crs=src.crs
                height=raster.shape[1]
                width=raster.shape[2]

            #write to disk as geotiff
            with rasterio.open(out_filename,
                               'w',
                               driver='GTiff',
                               height=height,
                               width=width,
                               count=1,
                               dtype=rasterio.float32,
                               crs=crs,
                               transform=transform
                              ) as dst:
                dst.write(raster)

        #open as xarray and write as cog
        da = assign_crs(xr.open_rasterio(out_filename))
        write_cog(da,
                  fname=out_filename,
                  overwrite=True)
    except:
        pass

### Download all datasets

In [ ]:
%%time
years = [str(i) for i in range(1981, 2022)]
months = [str(i).zfill(2) for i in range(1,13)]

for y in years:
    for m in months:
        print('working on: '+y+'-'+m,end='\r')
        
        download_and_cog_chirps(year=y,
                                month=m,
                                s3_dst='chirps_afr_monthly')

In [ ]:
# download_and_cog_chirps --date=2020-01 --s3-destination=s3://deafrica-input-datasets/chirps